In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from bs4 import BeautifulSoup


- define the input: a string containing the full paper
- choose a standard commonly used NLP library (https://www.stxnext.com/blog/top-python-nlp-libraries/) nltk // textblob
- add as many features as possible to each word of the dictionary
- look for the features that correlates the most with the 'count' <---- put any machine learning here 
- use those features to guess the difficulty on the article
- for the top N most difficult words you match the meaning with the API

In [34]:
dict_df = pd.read_csv("unigram_freq.csv")

In [35]:
dict_df[dict_df["word"] == "cnn"]

,word,count
6578,cnn,9199332


In [29]:
dict_df.head(5)

,word,count,difficulty_level,definition
0,the,23135851162,A1,used before nouns to refer to particular thing...
1,of,13151942776,A1,"used to show possession, belonging, or origin:"
2,and,12997637966,A1,"used to join two words, phrases, parts of sent..."
3,to,12136980858,A1,used before a verb to show that it is in the i...
4,a,9081174698,A1,the first letter of the English alphabet


In [36]:
dict_df["word"].sample(5)

37709                     kor
114044                essenes
246094    waitforsingleobject
35589               jellyfish
98311           chlorhexidine
Name: word, dtype: object

In [28]:
import random

In [45]:
pip install scrapfly-sdk

   ---------------------------------------- 0.0/62.5 kB ? eta -:--:--
   ---------------------------------------- 62.5/62.5 kB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
pip install shadow_useragent

     ---------------------------------------- 0.0/46.0 kB ? eta -:--:--
     -------- ------------------------------- 10.2/46.0 kB ? eta -:--:--
     ---------------- --------------------- 20.5/46.0 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 46.0/46.0 kB 328.2 kB/s eta 0:00:00
     ---------------------------------------- 0.0/86.8 kB ? eta -:--:--
     -------------- ------------------------- 30.7/86.8 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 86.8/86.8 kB 981.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
pip install torpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/90.7 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/90.7 kB ? eta -:--:--
     ------------ ------------------------- 30.7/90.7 kB 262.6 kB/s eta 0:00:01
     ----------------- -------------------- 41.0/90.7 kB 281.8 kB/s eta 0:00:01
     -------------------------------------- 90.7/90.7 kB 467.8 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torpy: filename=torpy-1.1.6-py3-none-any.whl size=84286 sha256=3949a620a08177cb5caf494f7821ef91650af64cabe9b79d9363d1f924338c48
  Stored in directory: c:\users\airad\appdata\local\pip\cache\wheels\df\63\8b\2942517ca245ffac8f6658c5687e8f1087d8caf85c828471c1
Successfully built torpy


In [41]:
def scrape_word(word):
    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36",
        "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:24.0) Gecko/20100101 Firefox/24.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/91.0.4472.114 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/78.0.3904.70 Safari/537.36",
        "Mozilla/5.0 (X11; Linux i586; rv:31.0) Gecko/20100101 Firefox/31.0",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    ]
    headers = {
        'User-Agent': random.choice(user_agents)
    }
    url = f"https://dictionary.cambridge.org/dictionary/english/{word}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:

        soup = BeautifulSoup(response.content, 'html.parser')
        definition_div = soup.find('div', class_='def ddef_d db')
        if definition_div:
            definition = definition_div.text

            difficulty_level = "Difficulty level not found"
            difficulty_span = soup.select_one('a[data-type]')
            if difficulty_span:
                difficulty_level = difficulty_span.get('data-type')
                return {"word": word, "difficulty_level": difficulty_level[0], "definition": definition}
            else:
                # Attempt to find difficulty level using the .gb.dgc class as a fallback
                difficulty_span_fallback = soup.select_one('.gc.dgc')
                if difficulty_span_fallback:
                    difficulty_level = difficulty_span_fallback.get_text(strip=True)
                    return {"word": word, "difficulty_level": difficulty_level[0], "definition": definition}


In [43]:
counter = 0
batch_size = 300
csv_filename = "dictionary_data.csv"  # Persistent location for the CSV file

# Iterate over the DataFrame
for index, row in dict_df.iterrows():
    word_data = scrape_word(row["word"])
    dict_df.loc[index, "difficulty_level"] = word_data["difficulty_level"]
    dict_df.loc[index, "definition"] = word_data["definition"]
    
    counter += 1  # Increment the counter after processing each word

    # Check if the counter has reached the batch size
    if counter >= batch_size or index == len(dict_df) - 1:
        # Save the current batch to the CSV file
        if index == batch_size - 1:
            # Write header only for the first batch
            dict_df.iloc[index + 1 - batch_size:index + 1].to_csv(csv_filename, mode='w', header=True, index=False)
        else:
            # Append without header for subsequent batches
            dict_df.iloc[index + 1 - batch_size:index + 1].to_csv(csv_filename, mode='a', header=False, index=False)
        
        counter = 0  # Reset the counter after saving the batch

# If there are remaining rows that were not saved due to the batch size, save them now
if counter > 0:
    dict_df.iloc[-counter:].to_csv(csv_filename, mode='a', header=False, index=False)

print(f"CSV file saved: {csv_filename}")

%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s %(message)s
%(asctime)s:%(levelname)s:%(name)s:%(filename)s:%(lineno)d:%(funcName)s %(message)s


UnboundLocalError: cannot access local variable 'r' where it is not associated with a value

In [ ]:
len(dict_df[dict_df["difficulty_level"].notna()])